In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_csv("t20_data.csv.zip", compression="zip")

In [3]:
data.sample(3)

,match_id,venue,toss_winner,toss_decision,batting_team,bowling_team,innings,runs_so_far,wickets_so_far,balls_faced,run_rate,target_runs,required_run_rate,winner
227765,287858,Newlands,England,bat,England,Zimbabwe,1,89,3,56,8.9000,NaN,NaN,England
846763,1426469,Al Amerat Cricket Ground Oman Cricket (Ministr...,Saudi Arabia,field,Saudi Arabia,Qatar,2,93,5,93,5.8125,154.0,13.555556,Qatar
858971,1463665,"ICC Academy, Dubai",Qatar,field,Qatar,Bahrain,2,9,0,7,4.5000,99.0,4.778761,Qatar


In [4]:
data["batting_team_wins"] = (data["winner"] == data["batting_team"]).astype(int)
data.sample(3)

,match_id,venue,toss_winner,toss_decision,batting_team,bowling_team,innings,runs_so_far,wickets_so_far,balls_faced,run_rate,target_runs,required_run_rate,winner,batting_team_wins
565159,1451303,Sano International Cricket Ground,Japan,field,Japan,Singapore,2,18,0,15,6.000000,54.0,2.057143,Japan,1
336654,1273733,Dubai International Cricket Stadium,Australia,field,Sri Lanka,Australia,1,134,5,105,7.376147,NaN,NaN,Australia,0
125940,1275268,"Moara Vlasiei Cricket Ground, Ilfov County",Romania,bat,Czech Republic,Romania,2,9,0,2,5.400000,212.0,10.322034,Romania,0


In [5]:
# Fill missing numeric with -1
numeric_cols = data.select_dtypes(include=[np.number]).columns
data[numeric_cols] = data[numeric_cols].fillna(-1)

In [6]:
data.sample(5)

,match_id,venue,toss_winner,toss_decision,batting_team,bowling_team,innings,runs_so_far,wickets_so_far,balls_faced,run_rate,target_runs,required_run_rate,winner,batting_team_wins
244463,1443788,"Guernsey Rovers Athletic Club Ground, Port Soif",Estonia,field,Estonia,Czech Republic,2,56,3,54,6.222222,111.0,5.000000,Estonia,1
242,1442989,Pallekele International Cricket Stadium,Sri Lanka,field,Sri Lanka,India,2,130,5,111,6.842105,138.0,5.333333,NaN,0
680325,936153,SuperSport Park,Sri Lanka,field,Sri Lanka,South Africa,2,100,6,54,11.111111,127.0,2.454545,South Africa,0
497365,1321269,Tikkurila Cricket Ground,Romania,bat,Romania,Serbia,1,43,3,33,6.972973,-1.0,-1.000000,Romania,1
610838,1479327,"Mission Road Ground, Mong Kok, Hong Kong",Nepal,field,Nepal,Kuwait,2,29,4,30,5.800000,175.0,9.733333,Kuwait,0


In [7]:
from sklearn.preprocessing import LabelEncoder

label_encoders = {}
for col in ['venue', 'toss_winner', 'toss_decision', 'batting_team', 'bowling_team']:
    le = LabelEncoder()
    
    # Fit on all unique values in the FULL dataset (df), not just training split
    all_values = data[col].dropna().unique().tolist()
    le.fit(all_values)
    
    # Transform the full column now
    data[col] = le.transform(data[col])
    
    # Save encoder
    label_encoders[col] = le


In [8]:
data.sample(3)

,match_id,venue,toss_winner,toss_decision,batting_team,bowling_team,innings,runs_so_far,wickets_so_far,balls_faced,run_rate,target_runs,required_run_rate,winner,batting_team_wins
263311,1158355,41,83,0,84,55,1,57,2,49,6.218182,-1.0,-1.000000,Sierra Leone,1
681990,1476155,100,63,0,38,64,2,53,1,62,4.818182,113.0,6.206897,Hong Kong,1
871116,635658,274,86,0,90,87,2,91,6,106,5.055556,116.0,10.714286,South Africa,0


In [9]:
data_inn1 = data[data["innings"] == 1].copy()
data_inn2 = data[data["innings"] == 2].copy()

# Features for innings 1 → drop target-related
features_inn1 = [col for col in data_inn1.columns if col not in [
    "match_id", "winner", "batting_team_wins", 
    "target_runs", "runs_required", "required_rr"
]]

# Features for innings 2 → keep all
features_inn2 = [col for col in data_inn2.columns if col not in [
    "match_id", "winner", "batting_team_wins"
]]

In [10]:
from sklearn.model_selection import GroupShuffleSplit

# Split so same match doesn't leak between train/test
gss1 = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx1, test_idx1 = next(gss1.split(data_inn1[features_inn1], data_inn1["batting_team_wins"], groups=data_inn1["match_id"]))

gss2 = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx2, test_idx2 = next(gss2.split(data_inn2[features_inn2], data_inn2["batting_team_wins"], groups=data_inn2["match_id"]))

X1_train, X1_test = data_inn1[features_inn1].iloc[train_idx1], data_inn1[features_inn1].iloc[test_idx1]
y1_train, y1_test = data_inn1["batting_team_wins"].iloc[train_idx1], data_inn1["batting_team_wins"].iloc[test_idx1]

X2_train, X2_test = data_inn2[features_inn2].iloc[train_idx2], data_inn2[features_inn2].iloc[test_idx2]
y2_train, y2_test = data_inn2["batting_team_wins"].iloc[train_idx2], data_inn2["batting_team_wins"].iloc[test_idx2]

print("Train size:", X1_train.shape, "Test size:", X1_test.shape)
print("Train size:", X2_train.shape, "Test size:", X2_test.shape)

Train size: (439135, 11) Test size: (109610, 11)
Train size: (374039, 12) Test size: (94765, 12)


In [11]:
from lightgbm import LGBMClassifier

# First innings model
model_inn1 = LGBMClassifier(
    objective="binary",
    learning_rate=0.002,
    num_leaves=31,
    n_estimators=1200,
    random_state=42
)

model_inn1.fit(
    X1_train, y1_train,
    eval_set=[(X1_test, y1_test)],
    eval_metric="auc",
)


[LightGBM] [Info] Number of positive: 218554, number of negative: 220581
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004329 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1195
[LightGBM] [Info] Number of data points in the train set: 439135, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.497692 -> initscore=-0.009232
[LightGBM] [Info] Start training from score -0.009232


LGBMClassifier(learning_rate=0.002, n_estimators=1200, objective='binary',
               random_state=42)

In [12]:
# Second innings model
model_inn2 = LGBMClassifier(
    objective="binary",
    learning_rate=0.002,
    num_leaves=31,
    n_estimators=1000,
    random_state=42
)

model_inn2.fit(
    X2_train, y2_train,
    eval_set=[(X2_test, y2_test)],
    eval_metric="auc",
)

[LightGBM] [Info] Number of positive: 169417, number of negative: 204622
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004569 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1687
[LightGBM] [Info] Number of data points in the train set: 374039, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.452939 -> initscore=-0.188801
[LightGBM] [Info] Start training from score -0.188801


LGBMClassifier(learning_rate=0.002, n_estimators=1000, objective='binary',
               random_state=42)

In [13]:
from sklearn.metrics import roc_auc_score, accuracy_score

# First innings
y1_pred_proba = model_inn1.predict_proba(X1_test)[:, 1]
y1_pred = (y1_pred_proba >= 0.5).astype(int)
print("First Innings AUC:", roc_auc_score(y1_test, y1_pred_proba))
print("First Innings Accuracy:", accuracy_score(y1_test, y1_pred))

# Second innings
y2_pred_proba = model_inn2.predict_proba(X2_test)[:, 1]
y2_pred = (y2_pred_proba >= 0.5).astype(int)
print("Second Innings AUC:", roc_auc_score(y2_test, y2_pred_proba))
print("Second Innings Accuracy:", accuracy_score(y2_test, y2_pred))

First Innings AUC: 0.7396501442297218
First Innings Accuracy: 0.6746191040963416
Second Innings AUC: 0.9124063503537123
Second Innings Accuracy: 0.8268981163931831


In [14]:
import pickle

model_bundle = {
    "model_inn1": model_inn1,
    "model_inn2": model_inn2,
    "label_encoders": label_encoders,
    "features_inn1": features_inn1,
    "features_inn2": features_inn2
}

with open("pipe.pkl", "wb") as f:
    pickle.dump(model_bundle, f)

print("✅ Saved two-model setup to pipe.pkl")


✅ Saved two-model setup to pipe.pkl


In [27]:
data.columns

Index(['match_id', 'venue', 'toss_winner', 'toss_decision', 'batting_team',
       'bowling_team', 'innings', 'runs_so_far', 'wickets_so_far',
       'balls_faced', 'run_rate', 'target_runs', 'required_run_rate', 'winner',
       'batting_team_wins', 'pp_runs', 'pp_wickets', 'mid_runs', 'mid_wickets',
       'death_runs', 'death_wickets', 'pp_runs_known', 'mid_runs_known',
       'death_runs_known'],
      dtype='object')

In [46]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

param_grid_logistic = {
    'C': [0.01, 0.1, 1],       # Regularization strength
    'penalty': ['l2'],            # Norm used in the penalization
    'solver': ['lbfgs', 'newton-cg', 'newton-cholesky', 'sag', 'saga']
}
inn1_model_logistic = LogisticRegression(max_iter=1000)
inn1_grid_search_logistic = GridSearchCV(inn1_model_logistic, param_grid_logistic, cv=3, scoring='roc_auc', n_jobs=-1)
inn1_grid_search_logistic.fit(X1_train, y1_train)

inn2_model_logistic = LogisticRegression(max_iter=1000)
inn2_grid_search_logistic = GridSearchCV(inn2_model_logistic, param_grid_logistic, cv=3, scoring='roc_auc', n_jobs=-1)
inn2_grid_search_logistic.fit(X2_train, y2_train)

print("Best Parameters (inning 1):", inn1_grid_search_logistic.best_params_)
print("Best Cross-validation Accuracy (inning 1):", inn1_grid_search_logistic.best_score_)

print("Best Parameters (inning 2):", inn2_grid_search_logistic.best_params_)
print("Best Cross-validation Accuracy (inning 2):", inn2_grid_search_logistic.best_score_)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/optimize/_linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/optimize.py:204: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/optimize/_linesearch.py:425: LineSearchWarning: Rounding errors prevent the line search from converging
  warn(msg, LineSearchWarning)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/optimize/_linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/optimi

Best Parameters (inning 1): {'C': 0.01, 'penalty': 'l2', 'solver': 'sag'}
Best Cross-validation Accuracy (inning 1): 0.7328448976686911
Best Parameters (inning 2): {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
Best Cross-validation Accuracy (inning 2): 0.9111782693816367


In [ ]:
# First innings
y1_pred_proba_logistic = inn1_grid_search_logistic.predict_proba(X1_test)[:, 1]
y1_pred_logistic = (y1_pred_proba_logistic >= 0.5).astype(int)
print("First Innings AUC:", roc_auc_score(y1_test, y1_pred_proba_logistic))
print("First Innings Accuracy:", accuracy_score(y1_test, y1_pred_logistic))

# Best Parameters (inning 1): {'C': 0.01, 'penalty': 'l2', 'solver': 'sag'}

# Second innings
y2_pred_proba_logistic = inn2_grid_search_logistic.predict_proba(X2_test)[:, 1]
y2_pred_logistic = (y2_pred_proba_logistic >= 0.5).astype(int)
print("Second Innings AUC:", roc_auc_score(y2_test, y2_pred_proba_logistic))
print("Second Innings Accuracy:", accuracy_score(y2_test, y2_pred_logistic))

# Best Parameters (inning 2): {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}

First Innings AUC: 0.7216775825908632
First Innings Accuracy: 0.6606514004196697
Second Innings AUC: 0.9113077265293417
Second Innings Accuracy: 0.8232153221125943


In [51]:
import xgboost as xgb

param_grid_xgb = {
    'n_estimators': [50, 100, 150],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.7, 1.0],
    'colsample_bytree': [0.7, 1.0]
}

inn1_model_xgb = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
inn1_grid_search_xgb = GridSearchCV(inn1_model_xgb, param_grid_xgb, cv=3, scoring='roc_auc', n_jobs=-1)
inn1_grid_search_xgb.fit(X1_train, y1_train)

inn2_model_xgb = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
inn2_grid_search_xgb = GridSearchCV(inn2_model_xgb, param_grid_xgb, cv=3, scoring='roc_auc', n_jobs=-1)
inn2_grid_search_xgb.fit(X2_train, y2_train)

print("Best Parameters (inning 1):", inn1_grid_search_xgb.best_params_)
print("Best Cross-validation Accuracy (inning 1):", inn1_grid_search_xgb.best_score_)

print("Best Parameters (inning 2):", inn2_grid_search_xgb.best_params_)
print("Best Cross-validation Accuracy (inning 2):", inn2_grid_search_xgb.best_score_)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [13:22:04] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [13:22:04] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [13:22:04] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [13:22:04] WARN

Best Parameters (inning 1): {'colsample_bytree': 0.7, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100, 'subsample': 0.7}
Best Cross-validation Accuracy (inning 1): 0.7601113976241507
Best Parameters (inning 2): {'colsample_bytree': 0.7, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 50, 'subsample': 0.7}
Best Cross-validation Accuracy (inning 2): 0.9156274799659169


In [52]:
# First innings
y1_pred_proba_xgb = inn1_grid_search_xgb.predict_proba(X1_test)[:, 1]
y1_pred_xgb = (y1_pred_proba_xgb >= 0.5).astype(int)
print("First Innings AUC:", roc_auc_score(y1_test, y1_pred_proba_xgb))
print("First Innings Accuracy:", accuracy_score(y1_test, y1_pred_xgb))

# Best Parameters (inning 1): {'colsample_bytree': 0.7, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100, 'subsample': 0.7}

# Second innings
y2_pred_proba_xgb = inn2_grid_search_xgb.predict_proba(X2_test)[:, 1]
y2_pred_xgb = (y2_pred_proba_xgb >= 0.5).astype(int)
print("Second Innings AUC:", roc_auc_score(y2_test, y2_pred_proba_xgb))
print("Second Innings Accuracy:", accuracy_score(y2_test, y2_pred_xgb))

# Best Parameters (inning 2): {'colsample_bytree': 0.7, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 50, 'subsample': 0.7}

First Innings AUC: 0.7623782905767983
First Innings Accuracy: 0.6896086123528875
Second Innings AUC: 0.9144360739700117
Second Innings Accuracy: 0.8263177333403683
